# Ragas Evaluation with Llama Stack - Demo [remote execution]

This notebook demonstrates how to use the Ragas out-of-tree provider.


## Run your Llama Stack distribution

Nuke any old distro config files you might have lying around (I find these get in the way whenever I change my `.env` variables): 
```bash
ls ~/.llama/distributions/
rm -r ~/.llama/distributions/<name-of-your-distro>
```

Then, run your llama stack server with:
```bash
dotenv run uv run llama stack run distribution/run.yaml
```




## Setup and Imports


In [1]:
# Install dev packages if not already installed
# !uv pip install -e ".[dev]"

import os
from datetime import datetime

import pandas as pd
from llama_stack_client import LlamaStackClient
from rich.pretty import pprint

from llama_stack_provider_ragas.constants import PROVIDER_ID_INLINE, PROVIDER_ID_REMOTE

/Users/dmanilof/src/llama-stack-provider-ragas/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Llama Stack Client Setup

- Make sure we have an inference model (model_type='llm')
- Make sure we have an embedding model (model_type='embedding')


In [2]:
# If usingf the remote provider, you will need ngrok to enable remote access to your Llama Stack server
# Otherwise, the base_url is just http://localhost:8321
client = LlamaStackClient(base_url=os.getenv("KUBEFLOW_LLAMA_STACK_URL"))
available_models = client.models.list()
assert any(model.model_type == "llm" for model in available_models)
assert any(model.model_type == "embedding" for model in available_models)

INFO:httpx:HTTP Request: GET https://b95f8e27088c.ngrok-free.app/v1/models "HTTP/1.1 200 OK"


## Dataset Preparation

Create a sample RAG evaluation dataset. In a real scenario, you would load your own dataset.


In [3]:
# Sample Ragas evaluation dataset
evaluation_data = [
    {
        "user_input": "What is the capital of France?",
        "response": "The capital of France is Paris.",
        "retrieved_contexts": [
            "Paris is the capital and most populous city of France."
        ],
        "reference": "Paris",
    },
    {
        "user_input": "Who invented the telephone?",
        "response": "Alexander Graham Bell invented the telephone in 1876.",
        "retrieved_contexts": [
            "Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone."
        ],
        "reference": "Alexander Graham Bell",
    },
    {
        "user_input": "What is photosynthesis?",
        "response": "Photosynthesis is the process by which plants convert sunlight into energy.",
        "retrieved_contexts": [
            "Photosynthesis is a process used by plants to convert light energy into chemical energy."
        ],
        "reference": "Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.",
    },
]

## Dataset Registration

Register the dataset with Llama Stack's Datasets API using the direct rows approach.


In [4]:
# De-register the dataset if it already exists
dataset_id = "ragas_demo_dataset"
try:
    client.datasets.unregister(dataset_id)
except Exception:
    pass

INFO:httpx:HTTP Request: DELETE https://b95f8e27088c.ngrok-free.app/v1beta/datasets/ragas_demo_dataset "HTTP/1.1 404 Not Found"


In [5]:
dataset_response = client.datasets.register(
    dataset_id=dataset_id,
    purpose="eval/question-answer",  # RAG evaluation purpose
    source={"type": "rows", "rows": evaluation_data},
    metadata={
        "provider_id": "localfs",  # seems there's a bug in datasets
        "description": "Sample RAG evaluation dataset for Ragas demo",
        "size": len(evaluation_data),
        "format": "ragas",
        "created_at": datetime.now().isoformat(),
    },
)
pprint(dataset_response)

INFO:httpx:HTTP Request: POST https://b95f8e27088c.ngrok-free.app/v1beta/datasets "HTTP/1.1 200 OK"


DatasetRegisterResponse(
│   identifier='ragas_demo_dataset',
│   metadata={
│   │   'provider_id': 'localfs',
│   │   'description': 'Sample RAG evaluation dataset for Ragas demo',
│   │   'size': 3.0,
│   │   'format': 'ragas',
│   │   'created_at': '2025-10-28T11:58:17.339885'
│   },
│   provider_id='localfs',
│   purpose='eval/question-answer',
│   source=SourceRowsDataSource(
│   │   rows=[
│   │   │   {
│   │   │   │   'user_input': 'What is the capital of France?',
│   │   │   │   'response': 'The capital of France is Paris.',
│   │   │   │   'retrieved_contexts': ['Paris is the capital and most populous city of France.'],
│   │   │   │   'reference': 'Paris'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'Who invented the telephone?',
│   │   │   │   'response': 'Alexander Graham Bell invented the telephone in 1876.',
│   │   │   │   'retrieved_contexts': [
│   │   │   │   │   'Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone.'
│   │   │   │   ],
│   │   │   │   'reference': 'Alexander Graham Bell'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'What is photosynthesis?',
│   │   │   │   'response': 'Photosynthesis is the process by which plants convert sunlight into energy.',
│   │   │   │   'retrieved_contexts': [
│   │   │   │   │   'Photosynthesis is a process used by plants to convert light energy into chemical energy.'
│   │   │   │   ],
│   │   │   │   'reference': 'Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.'
│   │   │   }
│   │   ],
│   │   type='rows'
│   ),
│   type='dataset',
│   provider_resource_id='ragas_demo_dataset',
│   owner=None
)

## Benchmark Registration

Register a benchmark that defines what metrics to use for evaluation.


In [6]:
# comment out the provider you don't want to run
benchmarks_providers = [
    ("ragas_demo_benchmark__inline", PROVIDER_ID_INLINE),
    ("ragas_demo_benchmark__remote", PROVIDER_ID_REMOTE),
]

for benchmark_id, provider_id in benchmarks_providers:
    benchmark_response = client.benchmarks.register(
        benchmark_id=benchmark_id,
        dataset_id=dataset_id,
        scoring_functions=[
            "answer_relevancy",  # How relevant is the answer to the question?
            # "context_precision",     # How precise are the retrieved contexts?
            # "faithfulness",          # How faithful is the answer to the contexts?
            # "context_recall",        # How much of the ground truth is covered by contexts?
            # "answer_correctness"  # How correct is the answer compared to ground truth?
        ],
        provider_id=provider_id,
        # metadata={
        #     "provider": "ragas",
        #     "version": "1.0",
        #     "metrics_count": len(ragas_metrics),
        #     "created_at": datetime.now().isoformat()
        # }
    )

pprint(benchmark_response)

INFO:httpx:HTTP Request: POST https://b95f8e27088c.ngrok-free.app/v1alpha/eval/benchmarks "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://b95f8e27088c.ngrok-free.app/v1alpha/eval/benchmarks "HTTP/1.1 200 OK"


None

In [7]:
benchmarks = client.benchmarks.list()
pprint(benchmarks)

INFO:httpx:HTTP Request: GET https://b95f8e27088c.ngrok-free.app/v1alpha/eval/benchmarks "HTTP/1.1 200 OK"


[
│   Benchmark(
│   │   dataset_id='ragas_demo_dataset',
│   │   identifier='ragas_demo_benchmark__inline',
│   │   metadata={},
│   │   provider_id='trustyai_ragas_inline',
│   │   scoring_functions=['answer_relevancy'],
│   │   type='benchmark',
│   │   provider_resource_id='ragas_demo_benchmark__inline'
│   ),
│   Benchmark(
│   │   dataset_id='ragas_demo_dataset',
│   │   identifier='ragas_demo_benchmark__remote',
│   │   metadata={},
│   │   provider_id='trustyai_ragas_remote',
│   │   scoring_functions=['answer_relevancy'],
│   │   type='benchmark',
│   │   provider_resource_id='ragas_demo_benchmark__remote'
│   )
]

## Evaluation Execution

Run the evaluation using our Ragas out-of-tree provider.


In [8]:
# Review settings in distributinon/run.yaml, eg., note that
# since we can't set the embedding model in the benchmark config,
# the embedding model is set in the distribution run.yaml file(all-MiniLM-L6-v2)

remote_job = client.alpha.eval.run_eval(
    benchmark_id="ragas_demo_benchmark__remote",
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": "ollama/granite3.3:2b",
            "sampling_params": {"temperature": 0.1, "max_tokens": 100},
        },
        "scoring_params": {},
        # "num_examples": 1,
    },
)
pprint(remote_job)

INFO:httpx:HTTP Request: POST https://b95f8e27088c.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__remote/jobs "HTTP/1.1 200 OK"


Job(
│   job_id='70b2e450-ad69-45b1-b6a0-9153f3b9a76a',
│   status='in_progress',
│   runtime_config={
│   │   'benchmark_config': {
│   │   │   'eval_candidate': {
│   │   │   │   'type': 'model',
│   │   │   │   'model': 'ollama/granite3.3:2b',
│   │   │   │   'sampling_params': {
│   │   │   │   │   'strategy': {'type': 'greedy'},
│   │   │   │   │   'max_tokens': 100,
│   │   │   │   │   'repetition_penalty': 1.0,
│   │   │   │   │   'stop': None
│   │   │   │   },
│   │   │   │   'system_message': None
│   │   │   },
│   │   │   'scoring_params': {},
│   │   │   'num_examples': None
│   │   },
│   │   'embedding_model': 'ollama/all-minilm:latest',
│   │   'benchmark': {
│   │   │   'identifier': 'ragas_demo_benchmark__remote',
│   │   │   'provider_resource_id': 'ragas_demo_benchmark__remote',
│   │   │   'provider_id': 'trustyai_ragas_remote',
│   │   │   'type': 'benchmark',
│   │   │   'dataset_id': 'ragas_demo_dataset',
│   │   │   'scoring_functions': ['answer_relevancy'],
│   │   │   'metadata': {}
│   │   },
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   },
│   │   'kubeflow_config': {
│   │   │   'results_s3_prefix': 's3://public-rhods/ragas-evaluation-pipeline',
│   │   │   's3_credentials_secret_name': 'aws-credentials',
│   │   │   'pipelines_endpoint': 'https://ds-pipeline-dspa-ragas-eval-v3.apps.rosa.diego-ragas.zch5.p3.openshiftapps.com',
│   │   │   'namespace': 'ragas-eval-v3',
│   │   │   'llama_stack_url': 'https://b95f8e27088c.ngrok-free.app',
│   │   │   'base_image': 'quay.io/diegosquayorg/my-ragas-provider-image:latest',
│   │   │   'pipelines_api_token': None
│   │   }
│   },
│   kubeflow_run_id='2bee5a99-b086-4c6d-8085-48f98b2a7774',
│   result=None
)

In [9]:
# Review settings in distributinon/run.yaml, eg., note that
# since we can't set the embedding model in the benchmark config,
# the embedding model is set in the distribution run.yaml file(all-MiniLM-L6-v2)

inline_job = client.alpha.eval.run_eval(
    benchmark_id="ragas_demo_benchmark__inline",
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": "ollama/granite3.3:2b",
            "sampling_params": {"temperature": 0.1, "max_tokens": 100},
        },
        "scoring_params": {},
        # "num_examples": 1,
    },
)
pprint(inline_job)

INFO:httpx:HTTP Request: POST https://b95f8e27088c.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__inline/jobs "HTTP/1.1 200 OK"


Job(
│   job_id='0',
│   status='in_progress',
│   result=None,
│   eval_config={
│   │   'embedding_model': 'ollama/all-minilm:latest',
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   }
│   }
)

## Results Display


In [10]:
# wait a bit for the job to complete
pprint(
    client.alpha.eval.jobs.status(
        benchmark_id="ragas_demo_benchmark__inline", job_id=inline_job.job_id
    )
)

INFO:httpx:HTTP Request: GET https://b95f8e27088c.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__inline/jobs/0 "HTTP/1.1 200 OK"


Job(
│   job_id='0',
│   status='in_progress',
│   result=None,
│   eval_config={
│   │   'embedding_model': 'ollama/all-minilm:latest',
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   }
│   }
)

In [11]:
# wait a bit for the job to complete
pprint(
    client.alpha.eval.jobs.status(
        benchmark_id="ragas_demo_benchmark__remote", job_id=remote_job.job_id
    )
)

INFO:httpx:HTTP Request: GET https://b95f8e27088c.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__remote/jobs/70b2e450-ad69-45b1-b6a0-9153f3b9a76a "HTTP/1.1 200 OK"


Job(
│   job_id='70b2e450-ad69-45b1-b6a0-9153f3b9a76a',
│   status='in_progress',
│   runtime_config={
│   │   'benchmark_config': {
│   │   │   'eval_candidate': {
│   │   │   │   'type': 'model',
│   │   │   │   'model': 'ollama/granite3.3:2b',
│   │   │   │   'sampling_params': {
│   │   │   │   │   'strategy': {'type': 'greedy'},
│   │   │   │   │   'max_tokens': 100,
│   │   │   │   │   'repetition_penalty': 1.0,
│   │   │   │   │   'stop': None
│   │   │   │   },
│   │   │   │   'system_message': None
│   │   │   },
│   │   │   'scoring_params': {},
│   │   │   'num_examples': None
│   │   },
│   │   'embedding_model': 'ollama/all-minilm:latest',
│   │   'benchmark': {
│   │   │   'identifier': 'ragas_demo_benchmark__remote',
│   │   │   'provider_resource_id': 'ragas_demo_benchmark__remote',
│   │   │   'provider_id': 'trustyai_ragas_remote',
│   │   │   'type': 'benchmark',
│   │   │   'dataset_id': 'ragas_demo_dataset',
│   │   │   'scoring_functions': ['answer_relevancy'],
│   │   │   'metadata': {}
│   │   },
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   },
│   │   'kubeflow_config': {
│   │   │   'results_s3_prefix': 's3://public-rhods/ragas-evaluation-pipeline',
│   │   │   's3_credentials_secret_name': 'aws-credentials',
│   │   │   'pipelines_endpoint': 'https://ds-pipeline-dspa-ragas-eval-v3.apps.rosa.diego-ragas.zch5.p3.openshiftapps.com',
│   │   │   'namespace': 'ragas-eval-v3',
│   │   │   'llama_stack_url': 'https://b95f8e27088c.ngrok-free.app',
│   │   │   'base_image': 'quay.io/diegosquayorg/my-ragas-provider-image:latest',
│   │   │   'pipelines_api_token': None
│   │   }
│   },
│   kubeflow_run_id='2bee5a99-b086-4c6d-8085-48f98b2a7774',
│   result=None
)

In [18]:
remote_results = client.alpha.eval.jobs.retrieve(
    benchmark_id="ragas_demo_benchmark__remote", job_id=remote_job.job_id
)
pprint(remote_results)

INFO:httpx:HTTP Request: GET https://b95f8e27088c.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__remote/jobs/70b2e450-ad69-45b1-b6a0-9153f3b9a76a/result "HTTP/1.1 200 OK"


EvaluateResponse(
│   generations=[
│   │   {
│   │   │   'user_input': 'What is the capital of France?',
│   │   │   'response': 'The capital of France is Paris.',
│   │   │   'retrieved_contexts': ['Paris is the capital and most populous city of France.'],
│   │   │   'reference': 'Paris'
│   │   },
│   │   {
│   │   │   'user_input': 'Who invented the telephone?',
│   │   │   'response': 'Alexander Graham Bell invented the telephone in 1876.',
│   │   │   'retrieved_contexts': [
│   │   │   │   'Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone.'
│   │   │   ],
│   │   │   'reference': 'Alexander Graham Bell'
│   │   },
│   │   {
│   │   │   'user_input': 'What is photosynthesis?',
│   │   │   'response': 'Photosynthesis is the process by which plants convert sunlight into energy.',
│   │   │   'retrieved_contexts': [
│   │   │   │   'Photosynthesis is a process used by plants to convert light energy into chemical energy.'
│   │   │   ],
│   │   │   'reference': 'Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.'
│   │   }
│   ],
│   scores={
│   │   'answer_relevancy': ScoringResult(
│   │   │   aggregated_results={
│   │   │   │   'average': 0.8885472168000002,
│   │   │   │   'count': 3.0,
│   │   │   │   'min': 0.7868883467000001,
│   │   │   │   'max': 0.9873642530000001
│   │   │   },
│   │   │   score_rows=[
│   │   │   │   {'score': 0.9873642530000001},
│   │   │   │   {'score': 0.8913890507000001},
│   │   │   │   {'score': 0.7868883467000001}
│   │   │   ]
│   │   )
│   }
)

In [17]:
inline_results = client.alpha.eval.jobs.retrieve(
    benchmark_id="ragas_demo_benchmark__inline", job_id=inline_job.job_id
)
pprint(inline_results)

INFO:httpx:HTTP Request: GET https://b95f8e27088c.ngrok-free.app/v1alpha/eval/benchmarks/ragas_demo_benchmark__inline/jobs/0/result "HTTP/1.1 200 OK"


EvaluateResponse(
│   generations=[
│   │   {
│   │   │   'user_input': 'What is the capital of France?',
│   │   │   'retrieved_contexts': ['Paris is the capital and most populous city of France.'],
│   │   │   'response': 'The capital of France is Paris.',
│   │   │   'reference': 'Paris'
│   │   },
│   │   {
│   │   │   'user_input': 'Who invented the telephone?',
│   │   │   'retrieved_contexts': [
│   │   │   │   'Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone.'
│   │   │   ],
│   │   │   'response': 'Alexander Graham Bell invented the telephone in 1876.',
│   │   │   'reference': 'Alexander Graham Bell'
│   │   },
│   │   {
│   │   │   'user_input': 'What is photosynthesis?',
│   │   │   'retrieved_contexts': [
│   │   │   │   'Photosynthesis is a process used by plants to convert light energy into chemical energy.'
│   │   │   ],
│   │   │   'response': 'Photosynthesis is the process by which plants convert sunlight into energy.',
│   │   │   'reference': 'Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.'
│   │   }
│   ],
│   scores={
│   │   'answer_relevancy': ScoringResult(
│   │   │   aggregated_results={'answer_relevancy': 0.9314807851969061},
│   │   │   score_rows=[
│   │   │   │   {'score': 0.9747285060082872},
│   │   │   │   {'score': 0.9013394319548492},
│   │   │   │   {'score': 0.9183744176275819}
│   │   │   ]
│   │   )
│   }
)

## Inline vs Remote Side-by-side

In [19]:
pd.DataFrame.from_dict(
    {
        "inline": [
            r["score"] for r in inline_results.scores["answer_relevancy"].score_rows
        ],
        "remote": [
            r["score"] for r in remote_results.scores["answer_relevancy"].score_rows
        ],
    },
).assign(diff=lambda df: df["remote"] - df["inline"])

inline    remote      diff
0  0.974729  0.987364  0.012636
1  0.901339  0.891389 -0.009950
2  0.918374  0.786888 -0.131486